In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
itp_id = 300

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
analysis_date_start = '2022-05-01'
analysis_date_end = '2022-05-31'

In [6]:
gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter((_.service_date >= analysis_date_start),
                 (_.service_date <= analysis_date_end))
        >> filter(_.is_in_service == True)
        >> collect()
    )

In [7]:
gtfs_daily

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,6242274318901953391,882298,3487,300,0,12010,2022-05-24,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,12,12,51240,53340,0.583333
1,4100255323842349957,992844467933662236,883626,3501,300,0,77710,2022-05-23,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,14,14,35880,36900,0.283333
2,4100255323842349957,2261863230332748366,882532,3489,300,0,10,2022-05-13,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,60420,61800,0.383333
3,4100255323842349957,-1416126210572631739,882561,3489,300,0,10,2022-05-19,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,30180,31800,0.450000
4,4100255323842349957,-4093705086200775082,882559,3489,300,0,10,2022-05-10,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,28980,30600,0.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39725,4100255323842349957,-7720565348741684097,881887,3485,300,0,10,2022-05-16,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,65700,69240,0.983333
39726,4100255323842349957,2813770288023396209,881939,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,75540,78480,0.816667
39727,4100255323842349957,6051719817107357845,881930,3485,300,0,10,2022-05-06,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,26580,30660,1.133333
39728,4100255323842349957,-2854982520547433387,881917,3485,300,0,10,2022-05-03,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,39840,43680,1.066667


In [8]:
gtfs_daily.service_date.min()

datetime.date(2022, 5, 1)

In [9]:
gtfs_daily.service_date.max()

datetime.date(2022, 5, 31)

In [10]:
rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.test_rt_trips`
        WHERE date BETWEEN '{analysis_date_start}' AND '{analysis_date_end}'
        """
    )

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [11]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-01,882742,1301
1,300,0,2022-05-01,882736,1301
2,300,0,2022-05-01,882739,1301
3,300,0,2022-05-01,882771,1301
4,300,0,2022-05-01,882761,1301
...,...,...,...,...,...
38896,300,0,2022-05-31,881885,5316
38897,300,0,2022-05-31,883406,5316
38898,300,0,2022-05-31,883414,5316
38899,300,0,2022-05-31,883385,5316


In [12]:
rt['str_len'] = rt.trip_id.str.len()

In [13]:
rt.str_len.value_counts()

6    38901
Name: str_len, dtype: int64

In [14]:
len(rt)

38901

In [15]:
rt.trip_id.nunique()

3054

In [16]:
len(gtfs_daily)

39730

In [17]:
gtfs_daily.trip_id.nunique()

3158

In [18]:
date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [19]:
# start_date= ['2022-05-01']
# end_date=['2022-05-31']

from datetime import date, timedelta
start_date = date(2022, 5, 1)
end_date = date(2022, 5, 31)

In [20]:
daterange = pd.date_range(start_date, end_date)

In [21]:
print(daterange)

DatetimeIndex(['2022-05-01', '2022-05-02', '2022-05-03', '2022-05-04',
               '2022-05-05', '2022-05-06', '2022-05-07', '2022-05-08',
               '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12',
               '2022-05-13', '2022-05-14', '2022-05-15', '2022-05-16',
               '2022-05-17', '2022-05-18', '2022-05-19', '2022-05-20',
               '2022-05-21', '2022-05-22', '2022-05-23', '2022-05-24',
               '2022-05-25', '2022-05-26', '2022-05-27', '2022-05-28',
               '2022-05-29', '2022-05-30', '2022-05-31'],
              dtype='datetime64[ns]', freq='D')


In [22]:
#above range does not work with function

In [23]:
dates = set()

def daterange2(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

for single_date in daterange2(start_date, end_date):
    dates.add(single_date.strftime('%Y-%m-%d'))

In [24]:
(dates)

{'2022-05-01',
 '2022-05-02',
 '2022-05-03',
 '2022-05-04',
 '2022-05-05',
 '2022-05-06',
 '2022-05-07',
 '2022-05-08',
 '2022-05-09',
 '2022-05-10',
 '2022-05-11',
 '2022-05-12',
 '2022-05-13',
 '2022-05-14',
 '2022-05-15',
 '2022-05-16',
 '2022-05-17',
 '2022-05-18',
 '2022-05-19',
 '2022-05-20',
 '2022-05-21',
 '2022-05-22',
 '2022-05-23',
 '2022-05-24',
 '2022-05-25',
 '2022-05-26',
 '2022-05-27',
 '2022-05-28',
 '2022-05-29',
 '2022-05-30',
 '2022-05-31'}

In [53]:
rt['date'] = pd.to_datetime(rt['date'])

In [55]:
gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [56]:
single_date2 = '2022-05-08'

(rt>>filter(_.date == single_date2))


,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len
8910,300,0,2022-05-08,882227,1303,6
8911,300,0,2022-05-08,882219,1303,6
8912,300,0,2022-05-08,882246,1303,6
8913,300,0,2022-05-08,882212,1303,6
8914,300,0,2022-05-08,882217,1303,6
...,...,...,...,...,...,...
9692,300,0,2022-05-08,881140,5317,6
9693,300,0,2022-05-08,881211,5317,6
9694,300,0,2022-05-08,881201,5317,6
9695,300,0,2022-05-08,881356,5317,6


In [59]:
def get_pct_ran_df2(itp_id, date_list):
    pcts = []
    for single_date in date_list:
        gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
        rt2 = (rt>>filter(_.date == single_date))
        sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
        pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
        pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
        pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
        pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
        pct_ran['len_sched'] = (len(gtfs_daily2))
        pct_ran['len_rt'] = (len(rt2))

        pcts.append(pct_ran)                                                    
        #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
    pcts = pd.concat(pcts)
    return pd.DataFrame(pcts)


In [60]:
get_pct_ran_df2(itp_id, dates)

,date,pct_trips_ran,n_have_rt,n_missing_rt,unmatched_rt,len_sched,len_rt
0,2022-05-05,0.893735,1455,0,173,1516,1455
0,2022-05-10,0.915946,1482,0,136,1516,1482
0,2022-05-07,0.479117,608,395,266,870,1003
0,2022-05-04,0.916769,1487,0,135,1516,1487
0,2022-05-06,0.891736,1392,0,169,1464,1392
0,2022-05-31,0.602144,1067,256,449,1516,1323
0,2022-05-25,0.896168,1450,0,168,1516,1450
0,2022-05-27,0.906590,1417,0,146,1466,1417
0,2022-05-26,0.924598,1496,0,122,1516,1496
0,2022-05-01,0.515152,527,252,244,770,779


In [30]:
# # getting errors for each date
# final_range = utils.get_pct_ran_df(dates, itp_id)

In [31]:
# final_range>>arrange(_.date)

In [32]:
# final_range.pct_trips_ran.mean()

In [33]:
# final_range.pct_trips_ran.median()

In [34]:
# import altair as alt

## Adding Columns

In [35]:
def get_pct_ran_df(date_list, itp_id):
    pcts = []
    for date in date_list: 
        sched_rt_df = (utils.find_ran_trips(itp_id, date))
        pct_ran = (utils.get_pct_ran(sched_rt_df, date))
        pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
        pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
        pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
        pcts.append(pct_ran)                                                    
        #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
    pcts = pd.concat(pcts)
    pct = pd.DataFrame(pcts)
    return pct


In [36]:
date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [37]:
get_pct_ran_df(date_list, itp_id)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/l

,date,pct_trips_ran,n_have_rt,n_missing_rt,unmatched_rt
0,2022-05-01,0.515152,527,252,244
0,2022-05-02,0.603614,1069,252,450
0,2022-05-31,0.602144,1067,256,449


In [38]:
# # getting errors for each date
# get_pct_ran_df(date_list, itp_id)

In [39]:
# test = (utils.find_ran_trips(itp_id, '2022-05-01'))

In [40]:
# test>>filter(_.have_rt!='both')

In [41]:
## reminder: left is scheduled and right is rt

In [42]:
# test.have_rt.value_counts()

In [43]:
# test>>filter(_.have_rt!='both')>>count(_.trip_id)>>filter(_.n>1)

In [44]:
# test>>filter(_.have_rt!='both')>>count(_.vehicle_id)>>filter(_.n>1)>>arrange(-_.n)

In [45]:
# unmatched_rt = test>>filter(_.have_rt=='right_only')

In [46]:
## seeing if there are any matches to updates

In [47]:
# updates = query_sql(f'''
# SELECT * FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
# WHERE date = '2022-05-01' and calitp_itp_id = {itp_id}
# LIMIT 1000
# ''')

In [48]:
# join = (pd.merge(unmatched_rt, updates, how='outer', on='trip_id', indicator='_updates'))

In [49]:
# join._updates.value_counts()

In [50]:
#no matches

In [51]:
# unmatched_rt.info()

In [52]:
# (unmatched_rt>>select(_.trip_id,
#                       _.calitp_itp_id_y,
#                       _.calitp_url_number_y,
#                       _.date,
#                       _.vehicle_id,
#                       _.str_len,
#                      _.have_rt))